In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter

# === Leer datos desde Excel ===
archivo_excel = "Datos_Ventas_Productos_A.xlsx"
df = pd.read_excel(archivo_excel, sheet_name=0)

# Convertir la columna 'Fecha' a tipo datetime (asumiendo formato dd/mm/yyyy)
df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True)

# Calcular Ingreso
df['Ingreso'] = df['Precio Unitario'] * df['Unidades Vendidas']

# === Función para promedio y coeficiente de variación ===
def resumen_estadistico(df, grupo):
    resumen = df.groupby(grupo)['Ingreso'].agg(['mean', 'std'])
    resumen['CV (%)'] = resumen['std'] / resumen['mean'] * 100
    resumen = resumen[['mean', 'CV (%)']].round(2)
    resumen.columns = ['Promedio', 'Coef. de Variación (%)']
    return resumen.reset_index()

# === Resumen por cliente y producto ===
resumen_cliente = resumen_estadistico(df, 'Cliente')
resumen_producto = resumen_estadistico(df, 'Producto')

# === Venta máxima y mínima === KPI
venta_max = df['Ingreso'].max()
venta_min = df['Ingreso'].min()
venta_max_fila = df[df['Ingreso'] == venta_max].iloc[0]
venta_min_fila = df[df['Ingreso'] == venta_min].iloc[0]

# === Mostrar en consola ===
print("\n📊 Resumen por Cliente:")
print(resumen_cliente)

print("\n📦 Resumen por Producto:")
print(resumen_producto)

print("\n📈 Indicadores de Venta:")
print(f"Venta Máxima: ${venta_max:.2f} - Cliente: {venta_max_fila['Cliente']} - Producto: {venta_max_fila['Producto']}")
print(f"Venta Mínima: ${venta_min:.2f} - Cliente: {venta_min_fila['Cliente']} - Producto: {venta_min_fila['Producto']}")

# === Visualización ===
sns.set_style("whitegrid")
formatter = FuncFormatter(lambda x, _: f"${x:,.0f}")

# Preparar datos agrupados
ventas_por_producto = df.groupby('Producto')['Ingreso'].sum().reset_index()
ventas_por_cliente = df.groupby('Cliente')['Ingreso'].sum().reset_index()
ventas_por_fecha = df.groupby('Fecha')['Ingreso'].sum().reset_index()

# === Crear gráficos ===
fig, axs = plt.subplots(1, 3, figsize=(15, 4))
fig.suptitle("Análisis de Ventas", fontsize=16)

# Barras más pequeñas por producto
sns.barplot(data=ventas_por_producto, x='Producto', y='Ingreso', hue='Producto',
            palette='Blues', legend=False, ax=axs[0])
axs[0].set_title("Por Producto")
axs[0].yaxis.set_major_formatter(formatter)
axs[0].set_xlabel('')
axs[0].set_ylabel('')

# Barras más pequeñas por cliente
sns.barplot(data=ventas_por_cliente, x='Cliente', y='Ingreso', hue='Cliente',
            palette='Greens', legend=False, ax=axs[1])
axs[1].set_title("Por Cliente")
axs[1].yaxis.set_major_formatter(formatter)
axs[1].set_xlabel('')
axs[1].set_ylabel('')

# Línea de tendencia temporal
sns.lineplot(data=ventas_por_fecha, x='Fecha', y='Ingreso', marker='o', ax=axs[2], color='darkorange')
axs[2].set_title("Tendencia Temporal")
axs[2].yaxis.set_major_formatter(formatter)
axs[2].tick_params(axis='x', rotation=45)
axs[2].set_xlabel('')
axs[2].set_ylabel('')

plt.tight_layout()
plt.subplots_adjust(top=0.85)
plt.show()

# === Exportar a Excel ===
with pd.ExcelWriter("dashboard_ventas.xlsx", engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='Ventas Detalladas', index=False)
    resumen_cliente.to_excel(writer, sheet_name='Resumen por Cliente', index=False)
    resumen_producto.to_excel(writer, sheet_name='Resumen por Producto', index=False)

print("\n✅ Exportado a 'dashboard_ventas.xlsx'")
